# Positional Encoding

In [2]:
import torch as t
from torch import nn
import typing
import einops
import numpy as np

In [3]:
class PositionalEncoding(nn.Module):

    def __init__(self, embedding_dim: int, max_seq_len: int = 5000):
        super().__init__()
        self.dim = embedding_dim
        self.length = max_seq_len

        # mostly copied. i understand this, just need to work on 
        # making more tensors and getting more exposure to methods of making tensors
        def P (delta):
            n = 10000 # hardcoded
            d = embedding_dim
            l = max_seq_len
            sin_array = np.sin(delta / n ** (2 * np.arange(d//2) / d))
            cos_array = np.cos(delta / n ** (2 * np.arange(d//2) / d))

            array = np.zeros(d)
            array[::2] = sin_array
            array[1::2] = cos_array

            return array

        tokenArray = []
        for i in range(max_seq_len):
            tokenArray[i] =P(i)
        
        self.multMax = tokenArray
        

    def forward(self, x: t.Tensor) -> t.Tensor:
        '''
        x: shape (batch, seq_len, embedding_dim)
        '''
        return x + self.addMat


# Attention Mechanism

In [51]:
def single_head_attention(Q: t.Tensor, K: t.Tensor, V: t.Tensor) -> t.Tensor:
    '''
    Should return the results of self-attention (see the "Self-Attention in Detail" section of the Illustrated Transformer).

    With this function, you can ignore masking.

    Q: shape (seq_len, head_size)
    K: shape (seq_len, head_size)
    V: shape (seq_len, value_size)

    Return: shape (seq_len, value_size)
    '''

    # second step - calculate a "score"
    score = Q@(K.T) # shape (seq_len,seq_len)
    # third step - divide score by dimensionality
    score = score / np.sqrt(Q.shape[-1])
    # fourth step - softmax
    print(score[0])
    score = nn.functional.softmax(score, dim=1)
    print(score[0])

    # fifth step - multiply each value vector by the softmax score
    z = score@V
    return z


In [52]:
# personal mini-test to see if its working correctly
Q = t.tensor([[1],[3],[4]],dtype=float)
K = t.tensor([[1],[3],[4]],dtype=float)
V = t.tensor([[1],[3],[4]],dtype=float)

print(single_head_attention(Q,K,V))

tensor([1., 3., 4.], dtype=torch.float64)
tensor([0.0351, 0.2595, 0.7054], dtype=torch.float64)
tensor([[3.6351],
        [3.9522],
        [3.9820]], dtype=torch.float64)
